In [ ]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio
import json
import pandas as pd
from datetime import datetime  # Change the import statement
from datetime import timedelta

# Define global variables for dataframes
student_df = None
engagement_df = None
ums_att_df = None
ums_avg_att_df = None

ums_upload_avg_att=None

# Edoofy app information
edoofy_base_url = "https://edoofa-portal.bubbleapps.io/api/1.1/obj"
edoofy_bearer_token = "2cde31d8f48919a2db1467cc06a56132"
edoofy_headers = {'Authorization': f'Bearer {edoofy_bearer_token}'}

# UMS app information
ums_base_url = "https://edoofa-ums-90164.bubbleapps.io/version-test/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}

# Asynchronous function to fetch data from a table
async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    total_fetched = 0

    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        print(f"Fetching {table} data from {base_url}... Cursor: {cursor}")

        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table}: {await response.text()}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)
            total_fetched += len(new_records)

            print(f"Fetched {len(new_records)} new records, Total fetched: {total_fetched}")

            cursor += 100

            if len(new_records) < 100:
                print(f"Exiting loop, fetched less than 100 records.")
                break

    df = pd.DataFrame(records)
    print(f"Fetched {len(df)} records for {table}.")
    return df


def map_students_to_engagement(student_df, engagement_df):
    # Ensure '_id' and 'EWYL-group-name' are in student_df
    if '_id' in student_df.columns and 'EWYL-group-name' in student_df.columns:
        student_id_to_EWYL = dict(zip(student_df['_id'], student_df['EWYL-group-name']))
        engagement_df['ewyl'] = engagement_df['student'].map(student_id_to_EWYL)
    else:
        print("Error: '_id' or 'EWYL-group-name' not in student_df columns")
    return engagement_df


print("P1 Success")


In [ ]:


def process_engagement_data(ums_att_df, engagement_df, student_df):
    # Create mapping from EWYL-group-name to KAM-group-name from student_df
    ewyl_to_kam_mapping = dict(zip(student_df['EWYL-group-name'], student_df['KAM-group-name']))

    # Add KAM-group-name column to the engagement dataframe
    engagement_df['KAM-group-name'] = engagement_df['ewyl'].map(ewyl_to_kam_mapping)

    # Find the latest date in ums_att_df and latest engagement date
    latest_ums_date = pd.to_datetime(ums_att_df['date'].max())
    print(latest_ums_date)
    latest_engagement_date = pd.to_datetime(engagement_df['engagement-date'].max())
    print(latest_engagement_date)
    # Filter engagement data for dates between latest_ums_date and latest_engagement_date
    filtered_engagement_df = latest_engagement_df[
        (pd.to_datetime(latest_engagement_df['engagement-date']) > latest_ums_date) &
        (pd.to_datetime(latest_engagement_df['engagement-date']) <= latest_engagement_date)
    ]
    #print(filtered_engagement_df)

    # Prepare data for the new DataFrame
    new_data = []
    for index, row in filtered_engagement_df.iterrows():
        new_row = {
            'admissions-group-name': row['KAM-group-name'],
            'ewyl-group-name': row['ewyl'],
            'attendance-type': row['engagement-type'],
            'present': row['daily-attendance'],
            'date': row['engagement-date']
        }
        new_data.append(new_row)

    # Create a new DataFrame
    new_df = pd.DataFrame(new_data, columns=['admissions-group-name', 'attendance-type', 'ewyl-group-name', 'present', 'date'])
    #print(new_df)
    return new_df

print("P2 Success")



def get_percentage(number_of_present, total_sessions):
    if total_sessions == 0:
        return 0  # Avoid division by zero
    percent = (number_of_present / total_sessions) * 100
    print(percent)
    return percent

# Function to check if a row exists for the current month for a student
async def check_if_row_exists(student, year, month, ums_avg_att_df):
    # Filtering the ums_avg_att_df to check if there's an existing record for the student
    existing_rows = ums_avg_att_df[
        (ums_avg_att_df['ewyl-group-name'] == student) &
        (ums_avg_att_df['year'] == year) &
        (ums_avg_att_df['month'] == month)
    ]

    if not existing_rows.empty:
        # Assuming '_id' is the column name for the row ID
        return existing_rows['_id'].iloc[0]
    else:
        return False

    
print("P3 Success")

# Function to post a new attendance summary
async def post_new_attendance_summary(session, base_url, headers, data):
    api_url = f"{base_url}/Attendance-Summary"
    async with session.post(api_url, headers=headers, json=data) as response:
        if response.status == 201:
            print("Successfully created a new row in Attendance-Summary.")
        else:
            print(f"Failed to create a new row: {await response.text()}")

# Function to patch an existing attendance summary
async def patch_attendance_summary(session, base_url, headers, row_id, data):
    api_url = f"{base_url}/Attendance-Summary/{row_id}"
    async with session.patch(api_url, headers=headers, json=data) as response:
        if response.status == 200:
            print("Successfully updated the row in Attendance-Summary.")
        else:
            print(f"Failed to update the row: {await response.text()}")

            
print("P4 Success")

async def process_avg_att(session, base_url, headers, combined_df, ums_avg_att_df, is_current_month):
    results_df = pd.DataFrame(columns=['student', 'attendance-percentage'])

    grouped = combined_df.groupby('ewyl-group-name')
    for student, group in grouped:
        total_sessions = len(group)
        number_of_present = group['present'].sum()
        attendance_percentage = get_percentage(number_of_present, total_sessions)
        
        print(f"Student (ewyl-group-name): {student}")
        print(f"Total Sessions: {total_sessions}, Number Present: {number_of_present}")
        print(f"Attendance Percentage: {attendance_percentage}")
###
        current_year = datetime.now().year
        current_month = datetime.now().strftime("%B")
        first_day_of_month = datetime(current_year, datetime.now().month, 1).strftime("%Y-%m-%d")

        # Fetch the 'KAM-group-name' for the student
        kam_group_name = student_df[student_df['EWYL-group-name'] == student]['KAM-group-name'].iloc[0]
        ewyl_group = student_df[student_df['EWYL-group-name'] == student]['EWYL-group-name'].iloc[0]
        # Calculate avg-att-percent-till-last-month
        past_records = combined_df[
            (combined_df['ewyl-group-name'] == student) &
            ((combined_df['date'] < first_day_of_month) | (combined_df['date'] == first_day_of_month))
        ]
        past_present = past_records['present'].sum()
        print(past_present)
        past_total = len(past_records)
        print(past_total)
        avg_att_percent_till_last_month = get_percentage(past_present, past_total)

        row_id = await check_if_row_exists(student, current_year, current_month, ums_avg_att_df)

        if row_id:
            await patch_attendance_summary(session, base_url, headers, row_id, {'attendance-percentage': attendance_percentage})
        else:
            new_data = {
                'ewyl-group-name': ewyl_group,
                'admissions-group-name': kam_group_name,
                'attendance-percentage': attendance_percentage,
                'avg-att-percent-till-last-month': avg_att_percent_till_last_month,
                'year': current_year,
                'month': current_month,
                'first-day-of-month': first_day_of_month
            }
            await post_new_attendance_summary(session, base_url, headers, new_data)

        new_row = pd.DataFrame({'student': [student], 'attendance-percentage': [attendance_percentage]})
        results_df = pd.concat([results_df, new_row], ignore_index=True)

    return results_df

print("P5 Success")



In [ ]:
async def main():
    global student_df, engagement_this_month_df, engagement_previous_months_df, ums_att_df, ums_avg_att_df
    
    async with aiohttp.ClientSession() as session:
        # Fetch Student table from Edoofy without constraints
        student_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Student")

        # Calculate the start and end date for the last 12 months (excluding this month)
        current_date = datetime.now()
        end_date = current_date.replace(day=1) - timedelta(days=1)
        start_date = end_date - timedelta(days=365)

        # Create constraints for engagement dataframe for this month
        this_month_start = current_date.replace(day=1)
        this_month_end = current_date
        engagement_this_month_constraints = [
            {'key': 'engagement-date', 'constraint_type': 'greater than', 'value': this_month_start.isoformat()},
            {'key': 'engagement-date', 'constraint_type': 'less than', 'value': this_month_end.isoformat()},
            {'key': 'student', 'constraint_type': 'equals', 'value': '1695736497533x818400363201798900'}
        ]

        # Create constraints for engagement dataframe for previous months
        engagement_previous_months_constraints = [
            {'key': 'engagement-date', 'constraint_type': 'greater than', 'value': start_date.isoformat()},
            {'key': 'engagement-date', 'constraint_type': 'less than', 'value': this_month_start.isoformat()},
            {'key': 'student', 'constraint_type': 'equals', 'value': '1695736497533x818400363201798900'}
        ]

        # Fetch Engagement table from Edoofy with constraints for this month
        engagement_this_month_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Engagement", constraints=engagement_this_month_constraints)

        # Fetch Engagement table from Edoofy with constraints for previous months
        engagement_previous_months_df = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Engagement", constraints=engagement_previous_months_constraints)

        # Fetch tables from UMS without constraints
        ums_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance")
        ums_avg_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance-Summary")

        # Apply mapping to engagement dataframes
        engagement_this_month_df = map_students_to_engagement(student_df, engagement_this_month_df)
        engagement_previous_months_df = map_students_to_engagement(student_df, engagement_previous_months_df)
        

        # Process engagement data
        processed_df = process_engagement_data(ums_att_df, engagement_df, student_df)
        #print(processed_df)
        # Process average attendance
        await process_avg_att(session, ums_base_url, ums_headers, processed_df, ums_avg_att_df, True)


await main()